## Ch03 협업 필터링 추천 시스템

아이템 기반 추천은 생각만큼 정확한 결과를 기대하기 힘들다.  
개선 방법으로 개인의 취향 기준으로 추천하는 것으로 협업 필터링(Collaborative Filtering : CF) 알고리즘이다.

### 3.1 협업 필터링의 원리

추천의 대상이 되는 사람과 취향이 비슷한 사람들을  찾아서 공통적으로 좋아하는 제품 서비스를 추천해준다는 아이디어

협업 필터링의 과정
1. 취향이 비슷한 사용자를 유사성을 계산하여 neighbor 그룹으로 분류한다.
2. 분류한 그룹의 인원이 가장 좋게 평가한 아이템의 평점 평균을 내어 값이 높은 아이템을 추천한다.

### 3.2 유사도지표

상관계수
+ 평가 자료가 연속값인 경우 이해하기 쉬운 유사도

코사인 유사도
+ 상관계수는 이해하기 쉽지만 늘 좋은 결과를 가져오지 못한다.
+ 협업 필터링에서 널리 쓰이는 유사도지표로 코사인 유사도가 있다.
+ 아이템을 하나의 차원으로 보고 평가값을 좌표로 하여 각 사용자의 평가값을 벡터로 해서 두 사용자 간의 벡터의 각도를 구할 수 있다.
+ 두 사용자의 평가값이 유사할수록 코사인 값이 크다는 것을 알 수 있다.

타니모토 계수
+ 데이터가 이진값이면 상관계수나 코사인 유사도를 사용할 수 없다.
+ 이 경우 타니모토 계수를 사용한다.
+ 이진수 데이터에 대해 협업 필터링에서 좋은 결과를 보여준다.
+ 타니모토 계수를 변형하여 자카드 계수로 사용하기도 한다.

### 3.3 기본 CF 알고리즘

이웃(neighbor)을 전체 사용자로 하여 모든 사용자의 평점을 가지고 예측한다.
1. 모든 사용자 간의 평가의 유사도를 계산한다.(상관계수, 코사인 유사도 등)
2. 추천 대상이 되는 사람과 다른 사용자의 유사도를 추출한다.
3. 현재 사용자가 평가하지 않은 모든 아이템에 대해 현재 사용자의 예상 평가값을 구한다.  
예상 평가값은 다른 사용자의 해당 아이템에 대한 평가를 현재 사용자와 그 사용자와의 유사도로 가중해서 평균을 낸다.
4. 아이템 중에서 예상 평가값이 가장 높은 N개의 아이템을 추천한다.

2장에서 사용한 코드를 가져와서 실행하고 그 이후부터 살펴본다.

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

# 데이터 읽어 오기 
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('./data/u.user', sep='|', names=u_cols, encoding='latin-1')
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('./data/u.item', sep='|', names=i_cols, encoding='latin-1')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('./data/u.data', sep='\t', names=r_cols, encoding='latin-1')

# timestamp 제거 
ratings = ratings.drop('timestamp', axis=1)
# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]

# train, test 데이터 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

# 정확도(RMSE)를 계산하는 함수 
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수 
def score(model):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# train 데이터로 Full matrix 구하기 
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

In [2]:
rating_matrix

movie_id,1,2,3,4,5,6,7,8,9,10,...,1672,1673,1674,1675,1676,1677,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# train set 의 모든 가능한 사용자 pair 의 Cosine similarities 계산
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index = rating_matrix.index, columns = rating_matrix.index)
user_similarity

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.109791,0.048186,0.016624,0.261564,0.362560,0.336050,0.247600,0.037835,0.311350,...,0.321788,0.078787,0.193727,0.076032,0.134860,0.068969,0.198473,0.096841,0.119134,0.273707
2,0.109791,1.000000,0.067652,0.188663,0.082040,0.191279,0.072571,0.098767,0.221328,0.151722,...,0.117967,0.169489,0.309951,0.314379,0.277860,0.162675,0.157793,0.093292,0.089524,0.073361
3,0.048186,0.067652,1.000000,0.281197,0.029341,0.044806,0.024799,0.097995,0.060950,0.044091,...,0.018970,0.058957,0.147829,0.008066,0.120194,0.015999,0.172487,0.107544,0.082178,0.000000
4,0.016624,0.188663,0.281197,1.000000,0.028680,0.038457,0.058463,0.150527,0.134051,0.077989,...,0.028974,0.000000,0.100523,0.144548,0.067846,0.039098,0.102136,0.058402,0.059503,0.015388
5,0.261564,0.082040,0.029341,0.028680,1.000000,0.169117,0.267530,0.242019,0.047323,0.169763,...,0.250136,0.090763,0.090859,0.082062,0.161353,0.041764,0.150087,0.143966,0.131196,0.247262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.068969,0.162675,0.015999,0.039098,0.041764,0.078976,0.091576,0.094891,0.055614,0.062071,...,0.047600,0.188285,0.202682,0.186448,0.329942,1.000000,0.066587,0.124297,0.022217,0.092887
940,0.198473,0.157793,0.172487,0.102136,0.150087,0.246103,0.250351,0.217920,0.132829,0.229418,...,0.236853,0.124917,0.154857,0.056966,0.142448,0.066587,1.000000,0.132015,0.209124,0.157240
941,0.096841,0.093292,0.107544,0.058402,0.143966,0.096520,0.061238,0.141742,0.136240,0.032051,...,0.059468,0.158036,0.255100,0.073291,0.300877,0.124297,0.132015,1.000000,0.066374,0.028608


In [4]:
# 주어진 영화의 가중평균을 계산하는 함수
def CF_simple(user_id, movie_id):
    if movie_id in rating_matrix:
        # 현재 사용자와 다른 사용자 간의 유사도(user_similarity)
        sim_scores = user_similarity[user_id].copy()
        # 현재 영화에 대한 모든 사용자의 rating값 가져오기
        movie_ratings = rating_matrix[movie_id].copy()
        # 현재 영화를 평가하지 않은 사용자의 index 가져오기
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        # 현재 영화를 평가하지 않은 사용자의 rating (null) 제거
        movie_ratings = movie_ratings.dropna()
        # 현재 영화를 평가하지 않은 사용자의 similarity 값 제거
        sim_scores = sim_scores.drop(none_rating_idx)
        # 현재 영화를 평가한 모든 사용자의 가중 평균값 구하기
        mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
    else:
        mean_rating = 3.0
    return mean_rating

# 정확도 계산
score(CF_simple)

1.0201148755196527

연습문제
1. 위 코드를 수정해서 코사인 유사도 대신에 피어슨 상관계수를 사용하는 코드를 작성하고 RMSE를 계산하시오.

In [5]:
from scipy.stats import pearson3
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity_pearson = matrix_dummy.T.corr(method='pearson')
user_similarity_pearson = pd.DataFrame(user_similarity_pearson, index = rating_matrix.index, columns = rating_matrix.index)
user_similarity_pearson

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.059789,0.000664,-0.018893,0.194380,0.292614,0.231170,0.210344,0.005268,0.239166,...,0.254370,0.038228,0.122204,0.036760,0.070995,0.021952,0.139296,0.067840,0.061526,0.206248
2,0.059789,1.000000,0.045387,0.175080,0.042675,0.151480,0.005657,0.075358,0.209001,0.111089,...,0.077252,0.151668,0.282098,0.300307,0.252704,0.142296,0.127984,0.078454,0.060923,0.032824
3,0.000664,0.045387,1.000000,0.270620,-0.007254,0.002211,-0.038279,0.077485,0.047498,0.003223,...,-0.021290,0.041182,0.117284,-0.010055,0.093298,-0.004997,0.147094,0.094693,0.056991,-0.038651
4,-0.018893,0.175080,0.270620,1.000000,0.002575,0.008016,0.017290,0.136853,0.125099,0.050746,...,0.000672,-0.013576,0.077812,0.133354,0.047558,0.024402,0.082633,0.048582,0.041114,-0.011758
5,0.194380,0.042675,-0.007254,0.002575,1.000000,0.101256,0.184637,0.212061,0.023097,0.104788,...,0.194210,0.060399,0.030169,0.052413,0.114618,0.004992,0.102216,0.123280,0.088188,0.194566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.021952,0.142296,-0.004997,0.024402,0.004992,0.037525,0.034590,0.073900,0.041819,0.021358,...,0.007997,0.172672,0.173838,0.171333,0.309535,1.000000,0.037044,0.111471,-0.005216,0.057772
940,0.139296,0.127984,0.147094,0.082633,0.102216,0.196357,0.183256,0.191757,0.115100,0.180449,...,0.190491,0.100705,0.108532,0.031502,0.102964,0.037044,1.000000,0.114087,0.176725,0.108876
941,0.067840,0.078454,0.094693,0.048582,0.123280,0.070263,0.021962,0.128424,0.127617,0.003599,...,0.033550,0.147071,0.239004,0.061570,0.287779,0.111471,0.114087,1.000000,0.048804,0.002947


In [6]:
# 주어진 영화의 가중평균을 계산하는 함수
def CF_simple(user_id, movie_id):
    if movie_id in rating_matrix:
        # 현재 사용자와 다른 사용자 간의 유사도(user_similarity)
        sim_scores = user_similarity_pearson[user_id].copy()
        # 현재 영화에 대한 모든 사용자의 rating값 가져오기
        movie_ratings = rating_matrix[movie_id].copy()
        # 현재 영화를 평가하지 않은 사용자의 index 가져오기
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        # 현재 영화를 평가하지 않은 사용자의 rating (null) 제거
        movie_ratings = movie_ratings.dropna()
        # 현재 영화를 평가하지 않은 사용자의 similarity 값 제거
        sim_scores = sim_scores.drop(none_rating_idx)
        # 현재 영화를 평가한 모든 사용자의 가중 평균값 구하기
        mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
    else:
        mean_rating = 3.0
    return mean_rating

# 정확도 계산
score(CF_simple)

1.033827511169568

### 3.4 이웃을 고려한 CF

단순 CF 알고리즘을 개선할 수 있는 한 가지 방법은 이웃(neighbor)을 전체 사용자로 하는 대신에 유사도가 높은 사람만을 이웃으로 선정해서 이웃의 크기를 줄이는 것이다.

이웃을 정하는 기준에는 크게 2가지 방법이 존재한다.
1. 이웃의 크기를 미리 정해놓고 추천 대상 사용자와 가장 유사한 K명을 선택하는 KNN 방법
2. 이웃의 크기 대신 유사도의 기준을 정해놓고 기준을 충족시키는 사용자를 이웃으로 정하는 Thresholding

Thresholding 방법이 KNN 보다 정확하지만 정해진 기준을 넘는 사용자가 없어서 추천을 하지 못하는 경우가 많기 때문에 KNN이 무난하게 많이 쓰인다.

In [8]:
# 모델별 RMSE 를 계산하는 함수
def score(model, neighbor_size = 0):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

In [9]:
# Neighbor size 를 정해서 예측치를 계산하는 함수
def cf_knn(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_matrix:
        # 현재 사용자와 다른 사용자 간의 similarity 가져오기
        sim_scores = user_similarity[user_id].copy()
        # 현재 영화에 대한 모든 사용자의 rating값 가져오기
        movie_ratings = rating_matrix[movie_id].copy()
        # 현재 영화를 평가하지 않은 사용자의 index 가져오기
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        # 현재 영화를 평가하지 않은 사용자의 rating (null) 제거
        movie_ratings = movie_ratings.drop(none_rating_idx)
        # 현재 영화를 평가하지 않은 사용자의 similarity값 제거
        sim_scores = sim_scores.drop(none_rating_idx)
        # Neighbor size 가 지정되지 않은 경우
        if neighbor_size == 0:
            # 현재 영화를 평가한 모든 사용자의 가중평균값 구하기
            mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
        # Neighbor size 가 지정된 경우
        else:
            # 해당 영화를 평가한 사용자가 최소 2명이 되는 경우에만 계산
            if len(sim_scores) > 1:
                # 지정된 neighbor size 값과 해당 영화를 평가한 총 사용자 수 중 작은 것으로 결정
                neighbor_size = min(neighbor_size, len(sim_scores))
                # array로 바꾸기 (argsort를 사용하기 위함)
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                # 유사도를 순서대로 정렬
                user_idx = np.argsort(sim_scores)
                # 유사도를 neighbor size 만큼 받기
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                # 영화 rating을 neighbor size 만큼 받기
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                # 최종 예측값 계산
                mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            else:
                mean_rating = 3.0
    else:
        mean_rating = 3.0
    return mean_rating

# 정확도 계산
score(cf_knn, neighbor_size = 30)

1.011296493805951

실제 추천을 받는 기능을 구현해보자.  
추천을 받을 사용자 id, 추천 아이템 수, 이웃 크기를 넘겨주면 추천 영화 리스트를 넘겨주는 코드이다.

In [16]:
### 주어진 사용자에 대해 추천받기
# 전체 데이터로 full matrix와 cosine_similarity 구하기
rating_matrix = ratings.pivot_table(values='rating', index='user_id', columns='movie_id')
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

def recommender(user, n_items = 10, neighbor_size = 20):
    # 현재 사용자의 모든 아이템에 대한 평점 계산
    predictions = []
    rated_index = rating_matrix.loc[user][rating_matrix.loc[user] > 0].index # 이미 평가한 영화 확인
    items = rating_matrix.loc[user].drop(rated_index)
    for item in items.index:
        predictions.append(cf_knn(user, item, neighbor_size)) # 예상평점 계산
    
    recommendations = pd.Series(data = predictions, index = items.index, dtype = float)
    recommendations = recommendations.sort_values(ascending=False)[:n_items]  # 예상평점이 가장 높은 영화 선택
    recommended_items = movies.loc[recommendations.index]['title']
    return recommended_items

recommender(user = 2, n_items=5, neighbor_size= 30)

movie_id
1189                              That Old Feeling (1997)
1293                     Ayn Rand: A Sense of Life (1997)
1500    Prisoner of the Mountains (Kavkazsky Plennik) ...
1467                                     Cure, The (1995)
318                       Everyone Says I Love You (1996)
Name: title, dtype: object

### 3.5 최적의 이웃 크기 결정

일반적으로 이웃 크기가 증가하면 추천 정확도가 증가하다가 어느 시점 이후 감소하는 형태를 보인다.  
즉 추천의 정확도를 최대로 하는 최적의 이웃 크기가 존재한다.

이웃 크기가 지나치게 크면 집단별 취향의 차이가 없어지고 best-seller 방식과 유사하게 된다.  
이웃 크기가 지나치게 작으면 현재 사용자와 취향의 유사도가 매우 높은 소수의 이웃의 평가에만 의존하게 되어 예측 신뢰도가 낮아진다.(overfitting)

최적의 이웃 크기를 찾는 코드를 살펴보자.

In [17]:
### 최적의 neighbor size 구하기
# train set 으로 full matrix 와 cosine similarity 구하기
rating_matrix = x_train.pivot_table(values = 'rating', index = 'user_id', columns = 'movie_id')
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)
for neighbor_size in [10, 20, 30, 40, 50, 60]:
    print("Neighbor size = %d : RMSE = %.4f" % (neighbor_size, score(cf_knn, neighbor_size)))

Neighbor size = 10 : RMSE = 1.0263
Neighbor size = 20 : RMSE = 1.0128
Neighbor size = 30 : RMSE = 1.0113
Neighbor size = 40 : RMSE = 1.0109
Neighbor size = 50 : RMSE = 1.0116
Neighbor size = 60 : RMSE = 1.0124


### 3.6 사용자의 평가경향을 고려한 CF

CF의 정확도를 더 개선시키는 방법 중 하나는 사용자의 평가경향을 고려하는 것이다.  
지금까지의 알고리즘은 사용자의 평가경향을 고려하지 않고 사용자의 평점을 그대로 예측치 계산에 사용하였다.  
이를 식으로 표시하면 다음과 같다.

$\large p_{a,i} = \frac{\sum^n_{u=1} w_{a,u} r_{u,i}}{\sum^n_{u=1} w_{a,u}}$  
$a$ : 사용자, $u$ : 이웃 사용자, $n$ : 이웃 사용자의 수  
$p_{a,i}$ : 아이템 $i$에 대한 사용자 $a$의 예상 평점  
$w_{a,u}$ : 사용자 $a$와 $u$의 유사도    
$r_{u,i}$ : 아이템 $i$에 대한 사용자 $u$의 평점

사용자의 평가경향을 고려해서 예측치를 조정한다면 정확도가 개선될 것이라고 예상되는데 이유는 크게 2가지이다.
1. 평점평균이 2인 A가 x라는 영화에 3.0 평점을 준 것과 평점평균이 4인 B가 x라는 영화에 3.0 평점을 준 것은 다른 의미로 해석되어야 한다.
2. 추천의 대상이 되는 사용자의 경우에도 그 사용자의 평가경향을 고려한 조정이 필요하다.  
추천의 대상 사용자의 평점평균이 3.0인데 예측치를 구하는데 사용된 이웃들의 평점평균이 4.0인 경우 예측값에서 현재 사용자의 평가경향이 고려되어야 한다.

사용자들의 평가경향을 고려하는 경우 예측치를 계산하는 방법을 수식으로 표시하면 다음과 같다.

$\large p_{a,i} = \bar r_a + \frac{\sum^n_{u=1} w_{a,u} (r_{u,i}- \bar r_u)}{\sum^n_{u=1} w_{a,u}}$  
$a$ : 사용자, $u$ : 이웃 사용자, $n$ : 이웃 사용자의 수  
$p_{a,i}$ : 아이템 $i$에 대한 사용자 $a$의 예상 평점  
$w_{a,u}$ : 사용자 $a$와 $u$의 유사도    
$r_{u,i}$ : 아이템 $i$에 대한 사용자 $u$의 평점  
$\bar r_a$ : 사용자 $a$의 전체 평점평균  
$\bar r_u$ : 사용자 $u$의 전체 평점평균

위 식을 바탕으로 예측치를 구하는 알고리즘의 단계는 다음과 같다.
1. 각 사용자의 평점평균을 구한다.
2. 각 아이템의 평점을 각 사용자의 평균에서의 차이로 변환한다. 평점과 평균의 차이를 평점편차라고 하자.
3. 평점편차를 사용하여 해당 사용자의 해당 아이템의 편차 예측값을 구한다. 구체적으로는 해당 사용자의 이웃을 구하고 이들 이웃의 해당 아이템에 대한 평점편차와 유사도를 가중평균한다.
4. 이 편차 예측값은 평균에서의 차이를 의미하기 때문에 실제 예측값으로 변환하기 위해 현 사용자의 평균에 이 편차 예측값을 더해준다.
5. 예측값을 구할 수 없는 경우 지금까지는 3.0을 할당했지만 이번에는 해당 사용자의 평점 평균으로 대체한다.

In [18]:
# train 데이터의 user의 rating 평균과 영화의 평점편차 계산
rating_mean = rating_matrix.mean(axis = 1)
rating_bias = (rating_matrix.T - rating_mean).T

def CF_knn_bias(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_bias:
        # 현 user와 다른 사용자 간의 유사도 가져오기
        sim_scores = user_similarity[user_id].copy()
        # 현 movie의 평점편차 가져오기
        movie_ratings = rating_bias[movie_id].copy()
        # 현 movie에 대한 rating이 없는 사용자 삭제
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        movie_ratings = movie_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)
        # Neighbor size 가 지정되지 않은 경우
        if neighbor_size == 0:
            # 편차로 예측값(편차 예측값) 계산
            prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            # 편차 예측값에 현 사용자의 평균 더하기
            prediction = prediction + rating_mean[user_id]
        
        # Neighbor size 가 지정된 경우
        else:
            # 해당 영화를 평가한 사용자가 최소 2명이 되는 경우에만 계산
            if len(sim_scores) > 1:
                # 지정된 neighbor size 값과 해당 영화를 평가한 총 사용자 수 중 작은 것으로 결정
                neighbor_size = min(neighbor_size, len(sim_scores))
                # array로 바꾸기 (argsort를 사용하기 위함)
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                # 유사도를 순서대로 정렬
                user_idx = np.argsort(sim_scores)
                # 유사도와 rating을 neighbor size만큼 받기
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                # 편차로 예측치 계산
                prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
                # 예측값에 현 사용자의 평균 더하기
                prediction = prediction + rating_mean[user_id]
            else:
                prediction = rating_mean[user_id]
    else:
        prediction = rating_mean[user_id]
    return prediction

score(CF_knn_bias, 30)

0.9435399492187678

연습문제
1. 사용자 ID를 지정하면 해당 사용자를 위해 5개의 영화를 추천하도록 하세요.

In [20]:
# Cosine similarity 계산
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

# 전체 데이터의 user의 rating 평균과 영화의 평점편차 계산 
rating_mean = rating_matrix.mean(axis=1)
rating_bias = (rating_matrix.T - rating_mean).T

# 추천하기
def recommender(user, n_itmes = 5):
    predictions = []
    rated_index = rating_matrix.loc[user][rating_matrix.loc[user] > 0].index # 이미 평가한 영화 확인
    items = rating_matrix.loc[user].drop(rated_index)
    for item in items.index:
        predictions.append(CF_knn_bias(user, item)) # 예상평점 계산
    recommendations = pd.Series(data = predictions, index = items.index, dtype = float)
    recommendations = recommendations.sort_values(ascending=False)[:n_itmes] # 예상평점이 가장 높은 영화 순으로 선택
    recommended_items = movies.loc[recommendations.index]['title']
    return recommended_items

# 영화 추천함수 부르기
recommender(2, 5)

movie_id
814                                   One Fine Day (1996)
1536                                          Cosi (1996)
1500    Prisoner of the Mountains (Kavkazsky Plennik) ...
1512                                        Sprung (1997)
1306        Carmen Miranda: Bananas Is My Business (1994)
Name: title, dtype: object

### 3.7 그 외의 CF 정확도 개선 방법

앞서 언급한 방법 외에 CF 정확성을 개선할 수 있는 방법으로 신뢰도 가중이 있다.  
신뢰도 가중이란 어떤 사용자A와 공통으로 평가한 아이템이 10개인 사용자와 공통으로 평가한 아이템이 2개인 사용자 중에서 공통 아이템이 많은 사용자와의 유사도에 더 큰 가중치를 주자는 것이다.

단 예측값은 매우 민감해서 계산식을 약간만 바꿔서 RMSE 값이 크게 변동하기 때문에 공통 아이템 수를 가중치로 직접 사용하면 성능이 떨어질 수 있다.  
이에 대안으로 신뢰도가 일정 이상인 사용자만을 예측치 계산에 사용하는 것이다. 즉 신뢰도가 일정값 이상인 사용자만 이웃 사용자로 활용한다.

In [25]:
# 사용자별 공통 평가 수 계산
rating_binary1 = np.array((rating_matrix > 0).astype(float))
rating_binary2 = rating_binary1.T
counts = np.dot(rating_binary1, rating_binary2)
counts = pd.DataFrame(counts, index = rating_matrix.index, columns = rating_matrix.index).fillna(0)

def CF_knn_bias_sig(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_bias:
        # 현 user와 다른 사용자 간의 유사도 가져오기
        sim_scores = user_similarity[user_id]
        # 현 movie의 평점편차 가져오기
        movie_ratings = rating_bias[movie_id]
        # 현 movie에 대한 rating이 없는 사용자 표시
        no_rating = movie_ratings.isnull()
        # 현 사용자와 다른 사용자간 공통 평가 아이템 수 가져오기 
        common_counts = counts[user_id]
        # 공통으로 평가한 영화의 수가 SIG_LEVEL보다 낮은 사용자 표시
        low_significance = common_counts < SIG_LEVEL
        # 평가를 안 하였거나, SIG_LEVEL이 기준 이하인 user 제거
        none_rating_idx = movie_ratings[no_rating | low_significance].index
        movie_ratings = movie_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)
        # Neighbor size가 지정되지 않은 경우        
        if neighbor_size == 0:
            # 편차로 예측값(편차 예측값) 계산
            prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            # 편차 예측값에 현 사용자의 평균 더하기
            prediction = prediction + rating_mean[user_id]
        # Neighbor size가 지정된 경우            
        else:
            # 해당 영화를 평가한 사용자가 최소 MIN_RATINGS 이상인 경우에만 계산            
            if len(sim_scores) > MIN_RATINGS:
                # 지정된 neighbor size 값과 해당 영화를 평가한 총사용자 수 중 작은 것으로 결정
                neighbor_size = min(neighbor_size, len(sim_scores))
                # array로 바꾸기 (argsort를 사용하기 위함)
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                # 유사도를 순서대로 정렬
                user_idx = np.argsort(sim_scores)
                # 유사도와 rating을 neighbor size만큼 받기
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                # 편차로 예측치 계산
                prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
                # 예측값에 현 사용자의 평균 더하기
                prediction = prediction + rating_mean[user_id]
            else:
                prediction = rating_mean[user_id]
    else:
        prediction = rating_mean[user_id]
    return prediction

SIG_LEVEL = 3
MIN_RATINGS = 2
score(CF_knn_bias_sig, 30)

0.9437115508288415

연습문제
1. 예측값이 1이하이면 1로, 5이상이면 5로 수정하는 코드를 작성해보자.

In [31]:
# 추천하기
def recommender(user, n_items=10, neighbor_size=20):
    # 현재 사용자의 모든 아이템에 대한 예상 평점 계산
    predictions = []
    rated_index = rating_matrix.loc[user][rating_matrix.loc[user] > 0].index    # 이미 평가한 영화 확인
    items = rating_matrix.loc[user].drop(rated_index)
    for item in items.index:
        if CF_knn_bias_sig(user, item, neighbor_size) <= 1:
            predictions.append(1)
        elif CF_knn_bias_sig(user, item, neighbor_size) >= 5:
            predictions.append(5)
        else:
            predictions.append(CF_knn_bias_sig(user, item, neighbor_size))
    recommendations = pd.Series(data=predictions, index=items.index, dtype=float)
    recommendations = recommendations.sort_values(ascending=False)[:n_items]    # 예상평점이 가장 높은 영화 선택
    recommended_items = movies.loc[recommendations.index]['title']
    return recommended_items

recommender(2, 5, 30)

movie_id
1405    When Night Is Falling (1995)
1449          Golden Earrings (1947)
1131        Feeling Minnesota (1996)
515                Local Hero (1983)
132        Gone with the Wind (1939)
Name: title, dtype: object

### 3.8 사용자 기반 CF와 아이템 기반 CF

지금까지는 사용자 기반 CF(UBCF) 를 사용하였는데 이번엔 아이템 기반 CF(IBCF) 를 살펴보자.  
IBCF는 사용자들의 평가 패턴을 바탕으로 아이템 간의 유사도를 계산해서 사용자의 특정 아이템에 대한 예측 평점을 계산하는 방식이다.

장단점 비교  
+ UBCF는 각 사용자별로 맞춤형 추천을 하기 때문에 데이터가 풍부한 경우 정확한 추천이 가능하다.
+ IBCF는 정확도는 떨어지지만 사용자별로 따로 계산을 하지 않기 때문에 계산이 빠르다는 장점이 있다.
+ UBCF는 정확할 때는 매우 정확하지만 매우 터무니없는 추천을 하는 경우도 있는데 IBCF는 그럴 위험이 적다.
+ UBCF는 업데이트를 자주해야하지만 IBCF는 업데이트를 자주 하지 않아도 된다.

데이터가 적고 각 사용자에 대한 충분한 정보가 있는 경우 UBCF가 알맞고 데이터가 크거나 사용자에 대한 충분한 정보가 없는 경우 IBCF가 알맞다고 할 수 있다.

IBCF 를 구현해보자.

In [36]:
# train, test 데이터 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

# 정확도(RMSE)를 계산하는 함수 
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수 
def score(model):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# train 데이터로 Full matrix 구하기  
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

# train 데이터로 Full matrix 구하기  
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

In [37]:
# train set의 모든 가능한 아이템 pair의 Cosine similarities 계산
from sklearn.metrics.pairwise import cosine_similarity
rating_matrix_t = np.transpose(rating_matrix)
matrix_dummy = rating_matrix_t.copy().fillna(0)
item_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
item_similarity = pd.DataFrame(item_similarity, index=rating_matrix_t.index, columns=rating_matrix_t.index)

# 주어진 영화의 (movie_id) 가중평균 rating을 계산하는 함수, 
# 가중치는 주어진 아이템과 다른 아이템 간의 유사도(item_similarity)
def CF_IBCF(user_id, movie_id):
    if movie_id in item_similarity:      # 현재 영화가 train set에 있는지 확인
        # 현재 영화와 다른 영화의 similarity 값 가져오기
        sim_scores = item_similarity[movie_id]
        # 현 사용자의 모든 rating 값 가져오기
        user_rating = rating_matrix_t[user_id]
        # 사용자가 평가하지 않은 영화 index 가져오기
        non_rating_idx = user_rating[user_rating.isnull()].index
        # 사용자가 평가하지 않은 영화 제거
        user_rating = user_rating.dropna()
        # 사용자가 평가하지 않은 영화의 similarity 값 제거
        sim_scores = sim_scores.drop(non_rating_idx)
        # 현 영화에 대한 예상 rating 계산, 가중치는 현 영화와 사용자가 평가한 영화의 유사도
        mean_rating = np.dot(sim_scores, user_rating) / sim_scores.sum()
    else:
        mean_rating = 3.0
    return mean_rating

# 정확도 계산
score(CF_IBCF)

1.0186891426302673

### 3.9 추천 시스템의 성과측정지표